In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import json as js

# KPI 1: ETA vs ATA

In [2]:
pickup_dropoff_file= pd.read_csv ('pickup_dropoff.csv', sep=',')
booking_file=pd.read_json('bookings.json')

In [3]:
booking_file.head()

booking_created_at_utc_0                            booking_id  \
0  2017-06-09T16:00:07.000+02:00  52814070-43e7-4c95-bdd6-fb4547bf3407   
1  2017-06-09T16:00:12.000+02:00  cb294029-6851-44be-88a9-b8ba11ed5ec8   
2  2017-06-09T16:00:20.000+02:00  de14e461-8db6-482a-a427-e1c81e508ca7   
3  2017-06-09T16:00:26.000+02:00  f2af9859-3248-4e39-a91a-efbc67178b03   
4  2017-06-09T16:00:12.000+02:00  cb294029-6851-44be-88a9-b8ba11ed5ec8   

              cancelled_at_utc_0 cancelled_by   event  passenger_count  \
0                                              create                2   
1                                              create                1   
2                                              create                3   
3                                              create                2   
4  2017-06-09T16:00:32.000+02:00     consumer  update                1   

      status  
0       open  
1       open  
2       open  
3       open  
4  cancelled

In [4]:
pickup_dropoff_file.head()

datetime_utc_0 actual_time_utc_0   postal  \
0  2017-06-09T16:00:21.000+02:00               NaN  10967.0   
1  2017-06-09T16:00:26.000+02:00               NaN  10967.0   
2  2017-06-09T16:00:26.000+02:00               NaN  10961.0   
3  2017-06-09T16:00:40.000+02:00               NaN  10961.0   
4  2017-06-09T16:00:51.000+02:00               NaN  10559.0   

                             booking_id           committed_time_utc_0  \
0  de14e461-8db6-482a-a427-e1c81e508ca7  2017-06-09T16:01:47.000+02:00   
1  de14e461-8db6-482a-a427-e1c81e508ca7  2017-06-09T16:01:47.000+02:00   
2  f2af9859-3248-4e39-a91a-efbc67178b03  2017-06-09T16:05:34.000+02:00   
3  f2af9859-3248-4e39-a91a-efbc67178b03  2017-06-09T16:05:34.000+02:00   
4  c1e85615-7518-45ee-96e1-7eb5903223a4  2017-06-09T16:15:19.000+02:00   

            estimated_time_utc_0 status    type  
0  2017-06-09T16:01:47.000+02:00   open  pickup  
1  2017-06-09T16:01:47.000+02:00   open  pickup  
2  2017-06-09T16:05:34.000+02:00   open  pickup  
3  2017-06-09T16:05:34.000+02:00   open  pickup  
4  2017-06-09T16:15:19.000+02:00   open  pickup

In [5]:
#Get rows when actual time exists only
actual_time_exist = pd.DataFrame( pickup_dropoff_file[pickup_dropoff_file.actual_time_utc_0.notnull()])
actual_time_exist['estimated_time_utc_0']=pd.to_datetime(actual_time_exist['estimated_time_utc_0'] )
actual_time_exist['actual_time_utc_0']=pd.to_datetime(actual_time_exist['actual_time_utc_0'])
#Get time difference to check ETA VS ATA
actual_time_exist['est_act_dif']=(actual_time_exist['estimated_time_utc_0']- actual_time_exist['actual_time_utc_0']).dt.total_seconds()/60
#actual_time_exist.head()

In [64]:
Total_Closed_Trips=actual_time_exist['booking_id'].unique().shape[0]

Total_Trips=pickup_dropoff_file['booking_id'].unique().shape[0]
Ratio_Closed_Trips=Total_Closed_Trips*100/Total_Trips
Ratio_Closed_Trips
#Total_Trips



73.35109926715523

In [7]:
#Get minimum entry of estimated time and max entry to get time span of the data in the file
pickup_dropoff_file.sort_values('committed_time_utc_0',ascending=True)
min_time=pickup_dropoff_file.estimated_time_utc_0[0]
max_time=pickup_dropoff_file.iloc[-1,5]


In [8]:
#Get time span of entries of pickup and dropoff, average of entry in a day
time_span=pd.DataFrame()
time_span['committed_time_utc_0']=pd.to_datetime(pickup_dropoff_file['committed_time_utc_0'] ).sort_values()

time_span['committed_time_span']=time_span.diff(1,0)
time_span['committed_time_span']=time_span['committed_time_span'].dt.total_seconds()/60
time_span_average=time_span.committed_time_span.mean()*60

time_span_average

17.474869843812602

In [56]:
#Get totals of late ATA, or early ATA or Exact ATA
ETA_less_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif < 0 ].type.groupby([actual_time_exist[actual_time_exist.est_act_dif<0].postal,actual_time_exist[actual_time_exist.est_act_dif<0].type]).count().sort_values(ascending=False)
ETA_greater_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif > 0 ].type.groupby([actual_time_exist[actual_time_exist.est_act_dif>0].postal,actual_time_exist[actual_time_exist.est_act_dif>0].type]).count().sort_values(ascending=False)
ETA_equal_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif == 0 ].type.groupby([actual_time_exist[actual_time_exist.est_act_dif==0].postal,actual_time_exist[actual_time_exist.est_act_dif==0].type]).count().sort_values(ascending=False)
#Special_Cases
Severe_ETA_less_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif < -10 ].type.groupby([actual_time_exist[actual_time_exist.est_act_dif<-10].postal,actual_time_exist[actual_time_exist.est_act_dif<0].type]).count().sort_values(ascending=False)
Severe_ETA_greater_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif > 10 ].type.groupby([actual_time_exist[actual_time_exist.est_act_dif>10].postal,actual_time_exist[actual_time_exist.est_act_dif>0].type]).count().sort_values(ascending=False)
#Cancelled Number due to delays
Severe_ETA_less_ATA_ByPostal_ByType=actual_time_exist[actual_time_exist.est_act_dif < 0 ][actual_time_exist.status.str.contains('cancelled')].type.groupby([actual_time_exist[actual_time_exist.est_act_dif<-10].postal,actual_time_exist[actual_time_exist.est_act_dif<0].type]).count().sort_values(ascending=False)

#[actual_time_exist.status.str.contains('cancelled')]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [47]:
#Get Mean of late ATA, or early ATA or Exact ATA
ETA_less_ATA_ByPostal_ByType_Mean=actual_time_exist[actual_time_exist.est_act_dif < 0 ].est_act_dif.groupby([actual_time_exist[actual_time_exist.est_act_dif<0].postal,actual_time_exist[actual_time_exist.est_act_dif<0].type]).mean().sort_values(ascending=False)
ETA_greater_ATA_ByPostal_ByType_Mean=actual_time_exist[actual_time_exist.est_act_dif > 0 ].est_act_dif.groupby([actual_time_exist[actual_time_exist.est_act_dif>0].postal,actual_time_exist[actual_time_exist.est_act_dif>0].type]).mean().sort_values(ascending=False)
ETA_equal_ATA_ByPostal_ByType_Mean=actual_time_exist[actual_time_exist.est_act_dif == 0 ].est_act_dif.groupby([actual_time_exist[actual_time_exist.est_act_dif==0].postal,actual_time_exist[actual_time_exist.est_act_dif==0].type]).mean().sort_values(ascending=False)


In [57]:
# Show numbers and Get Percentage from the total grouped by type (pickup or dropoff) and percentage of the late pickup or dropoff from the total.
ETA_vs_ATA=pd.DataFrame()
ETA_vs_ATA['ETA_less_ATA']=ETA_less_ATA_ByPostal_ByType
ETA_vs_ATA['ETA_greater_ATA']=ETA_greater_ATA_ByPostal_ByType
ETA_vs_ATA['ETA_equal_ATA']=ETA_equal_ATA_ByPostal_ByType
ETA_vs_ATA['ETA_less_ATA_10mins']=Severe_ETA_less_ATA_ByPostal_ByType
ETA_vs_ATA['ETA_greater_ATA_10mins']=Severe_ETA_greater_ATA_ByPostal_ByType
ETA_vs_ATA['ETA_less_ATA_Mean']=ETA_less_ATA_ByPostal_ByType_Mean
ETA_vs_ATA['ETA_greater_ATA_Mean']=ETA_greater_ATA_ByPostal_ByType_Mean
ETA_vs_ATA['ETA_equal_ATA_Mean']=ETA_equal_ATA_ByPostal_ByType_Mean
ETA_vs_ATA['Cancelled']=Severe_ETA_less_ATA_ByPostal_ByType
ETA_vs_ATA['Total'] = actual_time_exist.postal.groupby([actual_time_exist.postal,actual_time_exist.type]).count()
ETA_vs_ATA['less_pct_total'] = ETA_vs_ATA.ETA_less_ATA * 100 / ETA_vs_ATA.Total 
ETA_vs_ATA = ETA_vs_ATA.reset_index([0,1])
ETA_vs_ATA.sort_values('Total',ascending=False)

postal     type  ETA_less_ATA  ETA_greater_ATA  ETA_equal_ATA  \
0    10969.0   pickup            62             33.0            NaN   
1    10999.0   pickup            47             30.0            NaN   
2    10969.0  dropoff            37             38.0            NaN   
3    10999.0  dropoff            35             19.0            NaN   
4    10117.0   pickup            33             15.0            NaN   
5    10997.0  dropoff            30             17.0            NaN   
6    10997.0   pickup            29             17.0            1.0   
15   10245.0  dropoff            22             21.0            NaN   
7    10119.0   pickup            27             15.0            NaN   
10   10967.0  dropoff            24             15.0            NaN   
8    10785.0   pickup            25             12.0            NaN   
16   10245.0   pickup            20             16.0            NaN   
13   10963.0   pickup            23             12.0            NaN   
17   10437.0  dropoff            20             14.0            NaN   
19   10178.0   pickup            17             16.0            1.0   
12   10967.0   pickup            23             11.0            NaN   
9    10405.0   pickup            24              7.0            NaN   
11   10785.0  dropoff            23              8.0            NaN   
35   10243.0  dropoff            11             20.0            NaN   
21   10117.0  dropoff            15             16.0            NaN   
14   10243.0   pickup            22              8.0            NaN   
26   10405.0  dropoff            14             15.0            NaN   
27   10119.0  dropoff            13             16.0            NaN   
39   10789.0   pickup            10             17.0            NaN   
18   10961.0   pickup            18              9.0            NaN   
41   10115.0  dropoff            10             16.0            NaN   
24   10437.0   pickup            14             11.0            NaN   
20   12047.0   pickup            16              9.0            NaN   
22   10247.0   pickup            14             10.0            NaN   
29   12101.0  dropoff            12             11.0            NaN   
..       ...      ...           ...              ...            ...   
115  10555.0   pickup             1              4.0            NaN   
96   10829.0   pickup             3              2.0            NaN   
98   10823.0  dropoff             2              3.0            NaN   
109  10777.0  dropoff             2              3.0            NaN   
108  10439.0   pickup             2              3.0            NaN   
119  13347.0  dropoff             1              3.0            NaN   
114  10625.0  dropoff             1              3.0            NaN   
113  10589.0  dropoff             1              3.0            NaN   
106  10715.0   pickup             2              2.0            NaN   
88   12059.0   pickup             3              1.0            NaN   
94   10825.0  dropoff             3              NaN            NaN   
117  13353.0  dropoff             1              2.0            NaN   
102  13353.0   pickup             2              1.0            NaN   
101  12099.0  dropoff             2              1.0            NaN   
111  14057.0  dropoff             1              2.0            NaN   
89   10825.0   pickup             3              NaN            NaN   
99   10585.0  dropoff             2              1.0            NaN   
103  13357.0  dropoff             2              1.0            NaN   
122  10823.0   pickup             1              1.0            NaN   
120  10711.0   pickup             1              1.0            NaN   
118  10711.0  dropoff             1              1.0            NaN   
105  10781.0   pickup             2              NaN            NaN   
116  10625.0   pickup             1              1.0            NaN   
100  10781.0  dropoff             2              NaN            NaN   
112  13347.0   pickup         

In [70]:
#Detailed delays over 10 mins
Extreme_Delays=pd.DataFrame()
actual_time_exist[actual_time_exist.est_act_dif < -10 ]
Extreme_Delays['Postal']=actual_time_exist.postal[actual_time_exist.est_act_dif < -5 ][actual_time_exist.status.str.contains('cancelled')]
Extreme_Delays['Cancelled_Delays_Time']=actual_time_exist.est_act_dif[actual_time_exist.est_act_dif < -5 ][actual_time_exist.status.str.contains('cancelled')]
Extreme_Delays.sort_values('Cancelled_Delays_Time',ascending=True)

#actual_time_exist.est_act_dif[actual_time_exist.est_act_dif < 0 ][actual_time_exist.status.str.contains('cancelled')].count()

Postal  Cancelled_Delays_Time
6766  10969.0             -26.600000
4097  10785.0             -16.333333
2737  10997.0             -11.150000
6374  10827.0             -10.766667
56    10965.0             -10.150000
9334  10997.0              -9.883333
5010  10405.0              -9.750000
8386  10437.0              -9.483333
6344  10117.0              -9.266667
9855  10178.0              -9.183333
5392  12051.0              -9.000000
4771  10405.0              -8.766667
4221  10115.0              -7.950000
3092  10559.0              -6.333333
1465  10825.0              -6.000000
8179  10969.0              -5.916667
9543  10777.0              -5.750000
9943  10243.0              -5.750000
9844  10243.0              -5.333333

In [74]:
Total_Delays= actual_time_exist.type.groupby(actual_time_exist.type).count()
Total_Delays

type
dropoff    1012
pickup     1098
Name: type, dtype: int64

# KPI_2: Percentage of Cancelled Trips

In [77]:
#Cancelled Booking Number
Unique_Cancelled_Bookings=pickup_dropoff_file[pickup_dropoff_file.status.str.contains('cancelled')][pickup_dropoff_file.type.str.contains('pickup')]
Cancelled_Booking_count=Unique_Cancelled_Bookings.shape[0]



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [78]:
# percent cancelation out of total   
Booking_Count=booking_file['booking_id'].unique().shape[0]
Cancellation_Rate=booking_file[booking_file.status.str.contains('cancelled')]['booking_id'].unique().shape[0] * 100 /booking_file['booking_id'].unique().shape[0]
Cancellation_Rate

32.17854763491006

In [79]:
# booking_id is unique per cancelation per dirver
booking_file[booking_file.status.str.contains('cancelled')][booking_file.cancelled_by.str.contains('driver')]['booking_id'].unique().shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


(84,)

In [80]:
bookings_cancel_consumer = booking_file[booking_file.status.str.contains('cancelled')][booking_file.cancelled_by.str.contains('consumer')]['booking_id'].unique()
bookings_cancel_driver = booking_file[booking_file.status.str.contains('cancelled')][booking_file.cancelled_by.str.contains('driver')]['booking_id'].unique()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [81]:
cancelled_consumer = pd.DataFrame()
for i in range(0, len(bookings_cancel_consumer)):
    cancelled_consumer = cancelled_consumer.append( Unique_Cancelled_Bookings[Unique_Cancelled_Bookings['booking_id'].str.contains(bookings_cancel_consumer[i])] , ignore_index=True)
    
    
cancelled_driver = pd.DataFrame()
for i in range(0, len(bookings_cancel_driver)):
    cancelled_driver = cancelled_driver.append( Unique_Cancelled_Bookings[Unique_Cancelled_Bookings['booking_id'].str.contains(bookings_cancel_driver[i])] , ignore_index=True)
    

In [88]:
cancelledconsumer  = cancelled_consumer.booking_id.groupby(cancelled_consumer.postal).count().sort_values(ascending=False)
cancelleddriver = cancelled_driver.booking_id.groupby(cancelled_driver.postal).count().sort_values(ascending=False)

In [89]:
#Final Data fram of cancellation with ratio from total cancellation and total bookings
Cancellation=pd.DataFrame()
Cancellation['cancelled_driver'] = cancelleddriver
Cancellation['cancelled_consumer'] = cancelledconsumer
Cancellation['Percent_From_TotalCancellation']=cancelledconsumer*100/Cancelled_Booking_count
Cancellation['Percent_From_TotalBookings']=cancelledconsumer*100/Booking_Count
Cancellation = Cancellation.reset_index(0)

In [90]:
Cancellation.sort_values('Percent_From_TotalCancellation',ascending=False)

postal  cancelled_driver  cancelled_consumer  \
0   10969.0                10                42.0   
9   10999.0                 3                29.0   
5   10119.0                 4                21.0   
12  10117.0                 2                19.0   
3   10827.0                 5                15.0   
29  10967.0                 1                15.0   
1   10789.0                 7                15.0   
11  10963.0                 3                14.0   
2   10997.0                 5                13.0   
24  10178.0                 1                13.0   
14  10783.0                 2                13.0   
17  10961.0                 2                11.0   
10  10553.0                 3                11.0   
21  10245.0                 1                 9.0   
4   10785.0                 4                 9.0   
13  10559.0                 2                 9.0   
23  12047.0                 1                 9.0   
7   10243.0                 3                 8.0   
26  12045.0                 1                 7.0   
30  10555.0                 1                 7.0   
25  10179.0                 1                 6.0   
27  10437.0                 1                 6.0   
31  10719.0                 1                 6.0   
19  12043.0                 2                 5.0   
6   10707.0                 4                 4.0   
8   10405.0                 3                 4.0   
16  10115.0                 2                 3.0   
22  12049.0                 1                 2.0   
32  10777.0                 1                 2.0   
18  10965.0                 2                 1.0   
20  10823.0                 1                 1.0   
28  10825.0                 1                 1.0   
15  12051.0                 2                 NaN   
33  14057.0                 1                 NaN   

    Percent_From_TotalCancellation  Percent_From_TotalBookings  
0                         8.695652                    2.798135  
9                         6.004141                    1.932045  
5                         4.347826                    1.399067  
12                        3.933747                    1.265823  
3                         3.105590                    0.999334  
29                        3.105590                    0.999334  
1                         3.105590                    0.999334  
11                        2.898551                    0.932712  
2                         2.691511                    0.866089  
24                        2.691511                    0.866089  
14                        2.691511                    0.866089  
17                        2.277433                    0.732845  
10                        2.277433                    0.732845  
21                        1.863354                    0.599600  
4                         1.863354                    0.599600  
13                        1.863354                    0.599600  
23                        1.863354                    0.599600  
7                         1.656315                    0.532978  
26                        1.449275                    0.466356  
30                        1.449275                    0.466356  
25                        1.242236                    0.399734  
27                        1.242236                    0.399734  
31                        1.242236                    0.399734  
19                        1.035197                    0.333111  
6                         0.828157                    0.266489  
8                         0.828157                    0.266489  
16                        0.621118                    0.199867  
22                        0.414079                    0.133245  
32                        0.414079                    0.133245  
18                        0.207039                    0.066622  
20                        0.207039                    0.066622  
28                        0.207039                    0.066622  
15  

In [94]:
#Exporting values to excel to draw scatter graph
writer = pd.ExcelWriter('Plot_graph.xlsx', engine='xlsxwriter')
ETA_vs_ATA.to_excel(writer, sheet_name='Sheet1')
Extreme_Delays.to_excel(writer, sheet_name='Sheet2')
Cancellation.to_excel(writer, sheet_name='Sheet3')
writer.save()

# KPI 3: Percentage of Rejected Trips

In [86]:
#Not many or noticeable ratio of trips are rejected so I didn't work on it in the presentation
Unique_Rejected_Bookings=pickup_dropoff_file[pickup_dropoff_file.status.str.contains('rejected')][pickup_dropoff_file.type.str.contains('pickup')]
Rejected_Booking_count=Unique_Rejected_Bookings.shape[0]
Rejected_Booking_count

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


0

In [87]:
Booking_Count=booking_file['booking_id'].unique().shape[0]
Rejection_Rate=booking_file[booking_file.status.str.contains('rejected')]['booking_id'].unique().shape[0] * 100 /booking_file['booking_id'].unique().shape[0]
Rejection_Rate

0.06662225183211193

In [37]:
booking_file[booking_file.status.str.contains('rejected')]['booking_id'].unique().shape[0] 

1